# 프로젝트 가보

In [5]:
from pdb import post_mortem
from queue import Empty
from unittest import result
from flask import Flask, render_template, redirect, request, url_for, session
import cx_Oracle as db

app = Flask(__name__)

db_id = 'parky'
db_pw = 'qwer1234'
url = '192.168.21.187:1521/xe'

app.secret_key = "gabo"

@app.route('/join', methods=['POST'])
def join():
    print('회원가입 요청')
    id = request.form['id']
    pw = request.form['pw']
    email = request.form['email']
    name = request.form['name']
    nick = request.form['nick']
    phone = request.form['phone']
    birth = request.form['birth']
    gender = request.form['gender']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "insert into t_user(user_id, user_pw,user_name,user_nick,user_birth,user_phone,user_email,user_gender,user_startdate) values (:1,:2,:3,:4,:5,:6,:7,:8,sysdate)"
    curs.execute(sql,(id,pw,name,nick,birth,phone,email,gender))
    conn.commit()
    curs.close()
    conn.close()
    print(f'회원정보 : {id},{pw},{name},{nick},{birth},{phone},{email},{gender}')
    return '가입 완료'

@app.route('/login', methods=['POST'])
def login():
    print('로그인 시도')
    id = request.form['id']
    pw = request.form['pw']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select * from t_user where (user_id = :1) and (user_pw = :2)"
    curs.execute(sql,(id,pw))
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    try :
        ##print("{},{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3].isoformat()[:10],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8].isoformat()[:10],result[0][10],result[0][11],result[0][13]))
        print("{} 로그인".format(result[0][0]))
    
        if result[0][12] == 1:
            ## 블랙리스트
            return "블랙리스트"
        elif result[0][9] is not None:
            return "탈퇴회원"
        else :
            return "{}".format(result[0][0])
            ##return "{},{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3].isoformat()[:10],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8].isoformat()[:10],result[0][10],result[0][11],result[0][13])
    except :
        return "회원아님"

@app.route('/end', methods=['POST'])
def end():
    print('탈퇴 요청')
    id = request.form['id']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "update t_user set user_enddate = sysdate where user_id = :1"
    curs.execute(sql(id))
    conn.commit()
    curs.close()
    conn.close()
    
    return "탈퇴"

## 좋아요 클릭(첫클릭 insert)
@app.route('/like_first', methods=['POST'])
def like_first(user_id,hide_user):
    print('좋아요 클릭')
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "insert into t_like values(:1,:2,1)"
    curs.execute(sql,(hide_user,user_id))
    conn.commit()
    curs.close()
    conn.close()

    like_check()    
    return "좋아요"

## 좋아요 중복클릭 확인
@app.route('/like_check', methods=['POST'])
def like_check():
    print('좋아요 중복클릭')
    user_id = request.form['user_id']
    hide_user = request.form['hide_user']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    try:
        sql = "select like_click from t_like where like_take = :1 and like_give = :2"
        curs.execute(sql,(hide_user,user_id))
        result = curs.fetchall()
        curs.close()
        conn.close()
        
        return str(result[0][0])
    except:
        like_first(user_id,hide_user)

## 좋아요 클릭(up)
@app.route('/like_up', methods=['POST'])
def like_up():
    print('좋아요 up')
    user_id = request.form['user_id']
    hide_user = request.form['hide_user']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "update t_like set like_click = 1 where like_take = :1 and like_give = :2"
    curs.execute(sql,(hide_user,user_id))
    conn.commit()
    curs.close()
    conn.close()
    
    return "좋아요 up"

## 좋아요 클릭(down)
@app.route('/like_down', methods=['POST'])
def like_down():
    print('좋아요 down')
    user_id = request.form['user_id']
    hide_user = request.form['hide_user']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "update t_like set like_click = 0 where like_take = :1 and like_give = :2"
    curs.execute(sql,(hide_user,user_id))
    conn.commit()
    curs.close()
    conn.close()
    
    return "좋아요 down"


## mainpage에서 지도에 마커 갱신
@app.route('/mappage', methods=['POST'])
def mappage():
    print('마커갱신')

    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select t_id,t_category,t_keywords1,t_keywords2,t_keywords3,t_hideuser,t_finduser,t_approval,t_latitude,t_longitude,t_hidedate,(select sum(like_click) from t_like l, t_treasure t where l.like_take = t.t_hideuser) from t_treasure t where t_approval = 1"
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()

    t_table = ""
    
    for i in range(len(result)):
        for j in range(len(result[0])):
            if j ==11:
                t_table += str(result[i][j]) + "/"
            else:
                t_table += str(result[i][j]) + ","
        
    print('갱신완료')
    print(t_table)
    return t_table

    #보물의 모든 정보를 리턴
    # return "{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8],result[0][9],result[0][10],result[0][11])


## 근처 보물 리스트
@app.route('/nearlist', methods=['POST'])
def nearlist():
    print('근처 보물')

    user_id = request.form['user_id']
    user_lati = request.form['user_lati']
    user_longi = request.form['user_longi']

    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = """select t.t_id,t.t_category,t.t_keywords1,t.t_keywords2,t.t_keywords3,t.t_hideuser,t.t_finduser,t.t_approval,t.t_latitude,t.t_longitude,t.t_hidedate, 
    (select sum(l.like_click) from t_like l where t.t_hideuser = l.like_take) from t_treasure t where 
    (t_latitude < (:1+0.001000) and t_longitude < (:2+0.001000)) or (t_latitude < (:3-0.001000) and t_longitude < (:4-0.001000)) 
    or (t_latitude < (:5+0.001000) and t_longitude < (:6-0.001000)) or (t_latitude < (:7-0.001000) and t_longitude < (:8+0.001000))"""
    curs.execute(sql,(user_lati,user_longi,user_lati,user_longi,user_lati,user_longi,user_lati,user_longi))
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()


    t_list = ""
    for i in range(len(result)):
        for j in range(len(result[0])):
            if j ==11:
                t_list += str(result[i][j]) + "/"
            else:
                t_list += str(result[i][j]) + ","
    return t_list


## 보물 클릭시 정보 전송
@app.route('/clicktrs', methods=['POST'])
def clicktrs():
    print('보물클릭')

    t_id = request.form['t_id']

    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select t_category,t_keywords1,t_keywords2,t_keywords3,t_hideuser,t_finduser,t_latitude,t_longitude,t_hidedate,(select sum(like_click) from t_like l where (select t.t_hideuser from t_treasure t where t.t_id = :1) = l.like_take) from t_treasure t where t_id = :1"
    curs.execute(sql,(t_id))
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()


    return "{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8],result[0][9])

    #보물의 모든 정보를 리턴
    # return "{},{},{},{},{},{},{},{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3],result[0][4],result[0][5],result[0][6],result[0][7],result[0][8],result[0][9],result[0][10],result[0][11])


## 코멘트 저장(update)
@app.route('/addcomment', methods=['POST'])
def addcomment():
    t_id = request.form['t_id']
    find_user = request.form['find_user']
    t_comment = request.form['t_comment']
    print('코멘트 저장')
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "update t_treasure set t_comment = :1, t_finduser = :2, t_finddate = sysdate where t_id = :3"
    curs.execute(sql,(t_comment,find_user,t_id))
    conn.commit()
    curs.close()
    conn.close()

      
    return "코멘트등록"

## 보물 클릭시 올라오는 다이얼의 댓글리스트
@app.route('/commentlist', methods=['POST'])
def commentlist():
    print('댓글리스트업')

    hide_user = request.form['hide_user']
    empty_comment = 'null'

    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = """select t.t_finddate, t.t_finduser, t.t_keywords1, t.t_keywords2, t.t_keywords3, t.t_comment, (select sum(like_click) from t_like where like_take=t.t_finduser) 
    from t_treasure t where t_hideuser = :1 and t_comment != :2"""
    curs.execute(sql,(hide_user,empty_comment))
    result = curs.fetchall()
    curs.close()
    conn.close()

    t_comments = ""
    for i in range(len(result)):
        for j in range(len(result[0])):
            if j ==6:
                t_comments += str(result[i][j]) + "/"
            else:
                t_comments += str(result[i][j]) + ","
    ## 찾은 날짜, 찾은유저, 해쉬태그1 , 2 , 3 , 코멘트, 좋아요수
    print(t_comments)
    return t_comments

## 마이페이지 = 숨긴보물수, 찾은보물수, 좋아요, 레벨 갱신
@app.route('/mypage', methods=['POST'])
def mypage():
    print('마이페이지')
    id = request.form['id']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select (select count(t_hideuser) from t_treasure where t_hideuser = :1),(select count(t_finduser) from t_treasure where t_hideuser = :2),(select sum(like_click) from t_like l where l.like_take = :3),user_level from t_user where user_id = :4"
    curs.execute(sql,(id,id,id,id))
    result = curs.fetchall()
    
    curs.close()
    conn.close()
    print('갱신 성공')
    
    ## 숨긴보물수, 찾은보물수, 좋아요, 레벨 순
    return "{},{},{},{}".format(result[0][0],result[0][1],result[0][2],result[0][3])

    

@app.route('/alluserlist', methods=['POST'])
def alluserlist():
    print('모든 유저 리스트 조회')
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "select * from user"
    curs.execute(sql)
    result = curs.fetchall()
    print(result)
    curs.close()
    conn.close()
    
    listmember = ""
    
    for i in range(len(result)):
        for j in range(len(result[0])):
            if j ==3:
                listmember += str(result[i][j].isoformat()[:10]) + ","
            elif j ==13:
                listmember += str(result[i][j]) + "/"
            else:
                listmember += str(result[i][j]) + ","
        
        
    return listmember

@app.route('/addtreasure', methods=['POST'])
def addtreasure():
    print('보물 등록')
    cate = int(request.form['cate']) ## 코드(숫자)
    key1 = request.form['key1']
    key2 = request.form['key2']
    key3 = request.form['key3']
    hideuser = request.form['hideuser']
    latitude = float(request.form['latitude'])
    longitude = float(request.form['longitude'])
    
    img = request.form['img']
    
    conn = db.connect(db_id, db_pw, url)
    curs = conn.cursor()
    
    sql = "insert into t_treasure values(1,:1,:2,:3,:4,:5,'','',0,:6,:7,sysdate,'')"
    curs.execute(sql,(cate,key1,key2,key3,hideuser,latitude,longitude))
    conn.commit()
    curs.close()
    conn.close()
    
    return "등록신청완료"
    
#### 관리자 페이지

#로그인 페이지
@app.route('/', methods=['GET'])
def loginPage():
    return render_template('page-login.html')

#로그인 체크
@app.route('/loginCheck', methods = ['POST']) #로그인 기능
def loginCheck():
    print('로그인 요청')
    admin_id = request.form['id']
    admin_pw = request.form['pw']
    
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_admin where admin_id = :1 and admin_pw = :2"
    curs.execute(sql, (admin_id, admin_pw))
    result = curs.fetchall();

    curs.close()
    conn.close()
    print(admin_id)
    print(result[0][0])
    if result[0][0] ==  admin_id and result[0][1]==admin_pw:
        session['admin_id'] = admin_id
        return redirect(url_for('mainPage'))
    
# 회원가입 페이지
@app.route('/joinPage', methods=['GET'])
def joinPage():
    return render_template('page-join.html')


# 회원가입
@app.route('/joinAdmin', methods = ['POST'])
def joinAdmin():
    print('회원가입 요청')
    admin_id = request.form['id']
    admin_pw = request.form['pw']
    admin_name = request.form['name']
    print(admin_id)
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
#     sql = "insert into t_admin values('%s', '%s', '%s')" %(admin_id, admin_pw, admin_name)
    sql = "insert into t_admin values(:1, :2, :3)"
#     curs.execute(sql)
    curs.execute(sql,(admin_id,admin_pw,admin_name))

    conn.commit()
    curs.close()
    conn.close()
#     return redirect(url_for('/'))
    return render_template('page-login.html')

# 보물등록요청 리스트
@app.route('/tList', methods = ['GET'])
def tList():
    #login_user = session['login_user']
    print('등록리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select t_id, t_hideuser, t_hidedate from t_treasure where t_approval = 0"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            't_id' : obj[0],
            'hide_user' : obj[1],
            'hide_date' : obj[2]
        }
        data_list.append(data_dic)


    curs.close()
    conn.close()
    
    return render_template('tList.html',data_list=data_list)

# 유저 리스트
@app.route('/userList', methods = ['GET'])
def userList():
    print('유저 리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_user"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            'user_id' : obj[0],
            'user_name' : obj[2],
            'user_phone' : obj[4],
            'user_email' : obj[5],
            'user_startdate' : obj[8],
            'user_black' : obj[12]
        }
        data_list.append(data_dic)

    curs.close()
    conn.close()
    
    return render_template('userList.html',data_list=data_list)

# 블랙 리스트
@app.route('/blackList', methods = ['GET'])
def blackList():
    print('블랙리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_user where user_black = 0"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            'user_id' : obj[0],
            'user_name' : obj[2],
            'user_phone' : obj[4]
        }
        data_list.append(data_dic)

    curs.close()
    conn.close()
    
    return render_template('blackList.html',data_list=data_list)

#보물 승인
@app.route('/approveTreasure' , methods = ['GET'])
def approveTreasure():
    print('보물승인 요청')
    t_id = request.args['t_id']
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select t_hideuser, t_hidedate,t_latitude, t_longitude from t_treasure where t_id = "+str(t_id)
    curs.execute(sql)
    data = curs.fetchall()
    
    data_list = []
    
    for obj in data:
        data_dic = {
            't_hideuser' : obj[0],
            't_hidedate' : obj[1],
            't_latitude' : obj[2],
            't_longitude': obj[3]
        }
        data_list.append(data_dic)
    curs.close()
    conn.close()
    return render_template('checkTreasure.html', data_list = data_list)

#블랙리스트 on/off
@app.route('/applyBlack' , methods = ['GET'])
def applyBlack():
    print('블랙리스트 변경')
    user_id = request.args['user_id']
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = f"update t_user set user_black = 0 where user_id = '{user_id}'"
    curs.execute(sql)
    conn.commit()
    
    curs.close()
    conn.close()

    return redirect(url_for('userList'))

# 대쉬보드 
@app.route('/mainPage', methods = ['GET'])
def mainPage():
    print('보물갯수 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select (select count(t_approval) from t_treasure where t_approval = 0) as hide,(select count(t_approval) from t_treasure where t_approval = 1) as find, (select count(user_id) from t_user) as uu from t_treasure t, t_user u  where rownum = 1"
    curs.execute(sql)
    data = curs.fetchall()
    
    data_list = []

    for obj in data:
        data_dic = {
            'hideTreasure' : obj[0],
            'findTreasure' : obj[1],
            'userCount' : obj[2]
        }
        data_list.append(data_dic)
    print(data_list)
    
    curs.close()
    conn.close()
    
    return render_template('index.html',data_list = data_list)
          
          
if __name__ == '__main__':
    app.run(host='192.168.21.187', port=5013)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.187:5013/ (Press CTRL+C to quit)
192.168.20.215 - - [06/Sep/2022 16:20:20] "POST /clicktrs HTTP/1.1" 200 -
192.168.20.215 - - [06/Sep/2022 16:20:20] "POST /commentlist HTTP/1.1" 200 -
192.168.20.215 - - [06/Sep/2022 16:20:20] "POST /like_check HTTP/1.1" 200 -


보물클릭
[(2, '공원', '제일큰', '나무밑', 'pky5335', 'pky5335', 35.145158, 126.923793, datetime.datetime(2022, 9, 5, 14, 15, 9), 2)]
댓글리스트업
좋아요 중복클릭
2022-09-06 13:47:47,pky5335,cii,igo,igi,개쩜1,2,2022-09-06 13:47:51,test1,제일큰,나무밑,사무,개쩜3,None,2022-09-06 13:47:55,columbus,대성학,원3층,1강실,개쩜5,3,2022-09-06 13:47:57,pky5335,공원,제일큰,나무밑,개쩜7,2,2022-09-06 13:48:00,test1,나무,나무,나무,개쩜9,None,None,None,나무,나무,나무,개쩜10,None,


In [ ]:
!pip install cx_Oracle